In [1]:
import datetime

In [2]:
begin = datetime.date(2022, 3, 24)
end = datetime.date(2022, 4, 13)
delta = datetime.timedelta(days=1)
split_date = datetime.date(2022, 4, 7)

In [6]:
def generate_scope(d):
    
    cdate = d.strftime("%Y%m%d")
    
    f = open('scopes_2/{}.script'.format(cdate), 'w')

    print(cdate)
    cur_scripts = '''//Script 
#DECLARE Origin_data_path string = "/shares/bingads.hm/local/users/Yunfan/train_val/{cdate}.tsv";
#DECLARE Origin_header_path string = "/shares/bingads.hm/local/users/Yunfan/train_val/header.tsv";
RESOURCE @Origin_header_path;
#DECLARE Output_path string = "/shares/bingads.hm/local/users/Yunfan/samples_20/{cdate}.tsv";
#DECLARE ExpiryDuration string = "30";

originData = 
    EXTRACT * 
FROM @Origin_data_path
USING TSVWithHeaderExtractor("-schemaFile", @Origin_header_path);

output =
    SELECT * FROM originData SAMPLE UNIFORM (0.2);

OUTPUT output
TO @Output_path
    WITH STREAMEXPIRY @ExpiryDuration;

'''.format(cdate=cdate) + \
    '''
#CS
using System;
using System.Collections.Generic;
using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Linq;
using ScopeRuntime;


public class RandomUtils
{
    /// <summary>
    /// Get a random UInt64
    /// </summary>
    /// <returns>Random UInt64</returns>
    public static double GetRandomConsistentDoubleFromRGUID(string RGUID)
    {
        double rn = 0.99;
        Guid gd = new Guid();

        try
        {
            gd = Guid.Parse(RGUID);
        }
        catch (Exception e)
        {
            return rn;
        }

        byte[] bytes = gd.ToByteArray();
        int seed = BitConverter.ToInt32(bytes, 0);
        double rp = new Random(seed).NextDouble();

        return rp;
    }


}


public class TSVWithHeaderExtractor : Extractor
{
    private bool initialized = false;
    private string delim = "\t";
    private string schemaFile = "";
    private string schemaString = "";

       public static string LegalizeColumnName(string name)
    {
        string legalized = name.Replace("DateTime", "DateTime_");
        legalized = Regex.Replace(legalized, @"(^m:)", "m_");
        legalized = Regex.Replace(legalized, @"(@)", "_atsymbol_");

        // replace the illegal char brought in by Category transform and Generic transform
        legalized = Regex.Replace(legalized, @"([^\w\d_]+)", "_");

        // and cut the length to 450 if it's too long. C# has a 512 char length limit to identifier(so does scope)
        legalized = legalized.Substring(0, Math.Min(450, legalized.Length)); // cap length to 450

        return legalized;
    }

    public void Initialize(string[] args)
    {
        if (initialized)
            return;

        // create and add 
        // Process args, if any
        for (Int32 i = 0; i < args.Length; i++)
        {
            switch (args[i])
            {
                case "-schemaFile":
                    if (i == args.Length - 1)
                        throw new ArgumentException("No value specified", "schemaFile");
                    schemaFile = Path.GetFileName(args[++i]); //only get file name

                    // load up schemafile
                    using (StreamReader sr = new StreamReader(schemaFile))//read header
                    {
                        schemaString = string.Join(",", sr.ReadLine().Trim().Split('\t').Select(t => LegalizeColumnName(t)).ToArray());
                    }
                    break;

                case "-delim":
                    if (i == args.Length - 1)
                        throw new ArgumentException("No value specified", "delim");
                    delim = args[++i];
                    break;
            }
        }

        initialized = true;
    }

    public override Schema GetOutputSchemaAtCompileTime(string[] requestedColumns, string[] args)
    {
        Initialize(args);
        return new Schema(schemaString);
    }

    public override IEnumerable<Row> Extract(StreamReader reader, Row outputRow, string[] args)
    {
        Initialize(args);

        string line;
        int count = 0;

        while ((line = reader.ReadLine()) != null)
        {
            count++;

            if (count == 1 && (line.StartsWith("m:", StringComparison.OrdinalIgnoreCase) || line.StartsWith("Feature_", StringComparison.OrdinalIgnoreCase)))//skip header
            {
                continue;
            }

            string[] tokens = line.Split(delim.ToCharArray());
            if (tokens.Length < outputRow.Count)
            {
                continue;
            }

            try
            {
                for (int i = 0; i < tokens.Length; ++i)
                {
                    outputRow[i].UnsafeSet(tokens[i]);
                }
            }
            catch(Exception)
            {
                continue;
            }

            yield return outputRow;
        }
    }
}
public class HeaderOutputter : Outputter
{
       public static string RevertLegalizeColumnName(string name)
    {
        string legalized = name.Replace("DateTime_", "DateTime");
        legalized = Regex.Replace(legalized, @"(^m_)", "m:");
        legalized = Regex.Replace(legalized, @"(_atsymbol_)", "@");
        return legalized;
    }

       public override void Output(RowSet input, StreamWriter writer, string[] args)
       {
              // output the header
              writer.WriteLine(string.Join("\t", input.Schema.ToString().Split(',').Select(c => RevertLegalizeColumnName(c.Split(':')[0])).ToArray()));
              writer.Flush();
       }
}
#ENDCS
'''
    f.write(cur_scripts)
    f.close()

In [7]:
d = begin
while d <= end:
    generate_scope(d)
    d = d + delta

20220324
20220325
20220326
20220327
20220328
20220329
20220330
20220331
20220401
20220402
20220403
20220404
20220405
20220406
20220407
20220408
20220409
20220410
20220411
20220412
20220413
